In [19]:
## Initialize
import spacy
from spacy.language import Language
import requests
import json
import numpy as np
import re
from tqdm.notebook import trange, tqdm
  
apiKey = "hTLGygty2Sj5IB368rcArA63Xu29hW2r"
archiveUrl = f'https://api.nytimes.com/svc/archive/v1/#1/#2.json?api-key={apiKey}'

def parseDoc(doc) -> str:
  headline = doc["headline"]["print_headline"] or doc["headline"]["main"]
  leadPara = doc["lead_paragraph"]
  abstract = doc["abstract"]
  return str(
    headline 
    + (" " if leadPara else "") + leadPara
    + (" " if abstract else "") + abstract
  ).lower()

# Disabling components not needed (optional, but useful if run on a large dataset)
nlp = spacy.load("en_core_web_lg", disable=["tok2vec", "parser", "senter", "lemmatizer", "tagger", "attribute_ruler"])
nlp.add_pipe("merge_noun_chunks")
nlp.add_pipe("merge_entities")
tfidftfidf
# Example text
text = "German Chancellor Angela Merkel died in 1936 in New York. She got shot by a mysterious terrorist, terror, terrorism"

In [27]:
## Fetch

print(archiveUrl.replace("#1", "1963").replace("#2", "10"))
res = requests.get(archiveUrl.replace("#1", "1963").replace("#2", "10"))
obj = json.loads(res.text)
words = " ".join(map(lambda doc: parseDoc(doc), list(obj["response"]["docs"])))
# Test for clustering words in Caps
#words = "German Chancellor Angela Merkel died in 1936 in New York. She got shot by a mysterious terrorist, terror, terrorism. HIDE KILLED PAKISTAN STATEMENT MALAYSIA CONGRESS PLOT ALABAMA SUITS STATEMENT MALAYSIA CONGRESS Killed".lower()
words


https://api.nytimes.com/svc/archive/v1/1963/10.json?api-key=hTLGygty2Sj5IB368rcArA63Xu29hW2r


{'copyright': 'Copyright (c) 2020 The New York Times Company. All Rights Reserved.',
 'response': {'docs': [{'abstract': '',
    'web_url': 'https://www.nytimes.com/1963/10/01/archives/foreign-mutual-funds.html',
    'snippet': '',
    'lead_paragraph': '',
    'print_section': 'B',
    'print_page': '75',
    'source': 'The New York Times',
    'multimedia': [],
    'headline': {'main': 'Foreign Mutual Funds',
     'kicker': '1',
     'content_kicker': None,
     'print_headline': 'Foreign Mutual Funds',
     'name': None,
     'seo': None,
     'sub': None},
    'keywords': [],
    'pub_date': '1963-10-01T05:00:00+0000',
    'document_type': 'article',
    'news_desk': 'None',
    'section_name': 'Archives',
    'byline': {'original': '', 'person': [], 'organization': None},
    'type_of_material': 'Archives',
    '_id': 'nyt://article/000f4ebd-3609-5976-a472-8995618963ba',
    'word_count': 0,
    'uri': 'nyt://article/000f4ebd-3609-5976-a472-8995618963ba'},
   {'abstract': 'Pakista

In [21]:
## Search

def checkMatches(token, relevant_words):
    for comp in relevant_words.keys():
        comp_lg = nlp(comp)
        similarity = token.similarity(comp_lg)
        if(similarity >= 0.8):
            return (comp, similarity)
    return False

# Task 1: Extracting relevant words using the transformer-based model
wordCount = {}
matches: dict[str, dict] = {}
doc_trf = nlp(words)
for i in trange(len(list(doc_trf)), desc='Counting words'):
    token = doc_trf[i]
    if token.text in matches.keys():
        matches[token.text]["amount"] = matches[token.text]["amount"] + 1
    elif token.text in wordCount.keys():
        wordCount[token.text]["amount"] = wordCount[token.text]["amount"] + 1
    elif not token.is_stop and not token.is_punct:
        matched = checkMatches(token, wordCount)
        if(not matched):
            wordCount[token.text] = {}
            wordCount[token.text]["amount"] = 1
        else:
            matches[token.text] = {
                "match": matched[0],
                # This might be removed later
                "similarity": matched[1],
                "amount": 1
            }


1st loop:   0%|          | 0/143011 [00:00<?, ?it/s]

C:\Users\schwas28\AppData\Local\Temp\ipykernel_18468\4168163149.py:6: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity = token.similarity(comp_lg)


KeyboardInterrupt: 

In [22]:
for m in matches.items():
    if "duplicates" not in wordCount[m[1]["match"]]:
        wordCount[m[1]["match"]]["duplicates"] = {}
    wordCount[m[1]["match"]]["duplicates"][m[0]] = {
        "amount": m[1]["amount"],
        "similarity": m[1]["similarity"]
    }
    wordCount[m[1]["match"]]["amount"] = wordCount[m[1]["match"]]["amount"] + m[1]["amount"]

wordCount

{'foreign': {'amount': 2},
 'mutual': {'amount': 1},
 'funds': {'amount': 1},
 'pakistan': {'amount': 3,
  'duplicates': {'pakistani': {'amount': 1,
    'similarity': 0.8392587047835046}}},
 'pacts': {'amount': 1},
 '2': {'amount': 3},
 'red': {'amount': 2},
 'lands': {'amount': 1},
 'reaches': {'amount': 1},
 'china barter deal': {'amount': 2,
  'duplicates': {'deal': {'amount': 1, 'similarity': 0.9314840781640132}}},
 'soviet': {'amount': 5},
 'air': {'amount': 2},
 'accord': {'amount': 1},
 'barter': {'amount': 1},
 'nixon': {'amount': 1},
 'discloses': {'amount': 1},
 'rockefeller': {'amount': 1},
 'plan': {'amount': 1},
 'says': {'amount': 4,
  'duplicates': {'said': {'amount': 1, 'similarity': 0.8551111503321931}}},
 'governor': {'amount': 1},
 'clear': {'amount': 2},
 'seek': {'amount': 1},
 'nomination': {'amount': 1},
 'withholds': {'amount': 1},
 'support': {'amount': 1},
 'visited': {'amount': 1},
 'states': {'amount': 1},
 'kennedy': {'amount': 1},
 'pledges': {'amount': 1}

In [ ]:
print(wordCount)
print("\n--------------------\n")
print(matches)

{'german': {'amount': 1}, 'chancellor': {'amount': 1}, 'angela merkel': {'amount': 1}, 'died': {'amount': 1}, '1936': {'amount': 1}, 'new york': {'amount': 1}, 'got': {'amount': 1}, 'shot': {'amount': 1}, 'mysterious': {'amount': 1}, 'terrorist': {'amount': 3, 'duplicates': {'terror': {'amount': 1, 'similarity': 0.8488494070646385}, 'terrorism': {'amount': 1, 'similarity': 0.8807091504861693}}}, 'hide': {'amount': 1}, 'killed': {'amount': 2}, 'pakistan': {'amount': 1}, 'statement': {'amount': 2}, 'malaysia': {'amount': 1}, 'congress': {'amount': 2, 'duplicates': {'malaysia congress': {'amount': 1, 'similarity': 0.840037568990518}}}, 'plot': {'amount': 1}, 'alabama': {'amount': 1}, 'suits': {'amount': 1}}

--------------------

{'terror': {'match': 'terrorist', 'similarity': 0.8488494070646385, 'amount': 1}, 'terrorism': {'match': 'terrorist', 'similarity': 0.8807091504861693, 'amount': 1}, 'malaysia congress': {'match': 'congress', 'similarity': 0.840037568990518, 'amount': 1}}


In [ ]:
import spacy
taggers = [{
    "name": "Taggers",
    "values": [
        "$", "''", ",", "-LRB-", "-RRB-", ".", ":", "ADD", "AFX", "CC", "CD", "DT", "EX", "FW", "HYPH", "IN", "JJ", "JJR", "JJS", "LS", "MD", "NFP", "NN", "NNP", "NNPS", "NNS", "PDT", "POS", "PRP", "PRP$", "RB", "RBR", "RBS", "RP", "SYM", "TO", "UH", "VB", "VBD", "VBG", "VBN", "VBP", "VBZ", "WDT", "WP", "WP$", "WRB", "XX", "_SP", "``"
    ]
}]

for tagger in taggers:
    print(tagger["name"])
    print(tagger["values"])

for tagger in taggers:
    print(f"---------{tagger['name']}----------")
    for tag in tagger["values"]:
        print(f"{tag}: {spacy.explain(tag)}")

Taggers
['$', "''", ',', '-LRB-', '-RRB-', '.', ':', 'ADD', 'AFX', 'CC', 'CD', 'DT', 'EX', 'FW', 'HYPH', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NFP', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', 'XX', '_SP', '``']
---------Taggers----------
$: symbol, currency
'': closing quotation mark
,: punctuation mark, comma
-LRB-: left round bracket
-RRB-: right round bracket
.: punctuation mark, sentence closer
:: punctuation mark, colon or ellipsis
ADD: email
AFX: affix
CC: conjunction, coordinating
CD: cardinal number
DT: determiner
EX: existential there
FW: foreign word
HYPH: punctuation mark, hyphen
IN: conjunction, subordinating or preposition
JJ: adjective (English), other noun-modifier (Chinese)
JJR: adjective, comparative
JJS: adjective, superlative
LS: list item marker
MD: verb, modal auxiliary
NFP: superfluous punctuation
NN: noun, singular or mass
NNP: noun, pro

In [25]:
sections =  [
    {
      "section": "admin",
      "display_name": "Admin"
    },
    {
      "section": "arts",
      "display_name": "Arts"
    },
    {
      "section": "automobiles",
      "display_name": "Automobiles"
    },
    {
      "section": "books",
      "display_name": "Books"
    },
    {
      "section": "briefing",
      "display_name": "Briefing"
    },
    {
      "section": "business",
      "display_name": "Business"
    },
    {
      "section": "climate",
      "display_name": "Climate"
    },
    {
      "section": "corrections",
      "display_name": "Corrections"
    },
    {
      "section": "education",
      "display_name": "Education"
    },
    {
      "section": "en español",
      "display_name": "En español"
    },
    {
      "section": "fashion",
      "display_name": "Fashion"
    },
    {
      "section": "food",
      "display_name": "Food"
    },
    {
      "section": "gameplay",
      "display_name": "Gameplay"
    },
    {
      "section": "guide",
      "display_name": "Guide"
    },
    {
      "section": "health",
      "display_name": "Health"
    },
    {
      "section": "home & garden",
      "display_name": "Home & Garden"
    },
    {
      "section": "home page",
      "display_name": "Home Page"
    },
    {
      "section": "job market",
      "display_name": "Job Market"
    },
    {
      "section": "the learning network",
      "display_name": "The Learning Network"
    },
    {
      "section": "lens",
      "display_name": "Lens"
    },
    {
      "section": "magazine",
      "display_name": "Magazine"
    },
    {
      "section": "movies",
      "display_name": "Movies"
    },
    {
      "section": "multimedia/photos",
      "display_name": "Multimedia/Photos"
    },
    {
      "section": "new york",
      "display_name": "New York"
    },
    {
      "section": "obituaries",
      "display_name": "Obituaries"
    },
    {
      "section": "opinion",
      "display_name": "Opinion"
    },
    {
      "section": "parenting",
      "display_name": "Parenting"
    },
    {
      "section": "podcasts",
      "display_name": "Podcasts"
    },
    {
      "section": "reader center",
      "display_name": "Reader Center"
    },
    {
      "section": "real estate",
      "display_name": "Real Estate"
    },
    {
      "section": "smarter living",
      "display_name": "Smarter Living"
    },
    {
      "section": "science",
      "display_name": "Science"
    },
    {
      "section": "sports",
      "display_name": "Sports"
    },
    {
      "section": "style",
      "display_name": "Style"
    },
    {
      "section": "sunday review",
      "display_name": "Sunday Review"
    },
    {
      "section": "t brand",
      "display_name": "T Brand"
    },
    {
      "section": "t magazine",
      "display_name": "T Magazine"
    },
    {
      "section": "technology",
      "display_name": "Technology"
    },
    {
      "section": "theater",
      "display_name": "Theater"
    },
    {
      "section": "times insider",
      "display_name": "Times Insider"
    },
    {
      "section": "today’s paper",
      "display_name": "Today’s Paper"
    },
    {
      "section": "travel",
      "display_name": "Travel"
    },
    {
      "section": "u.s.",
      "display_name": "U.S."
    },
    {
      "section": "universal",
      "display_name": "Universal"
    },
    {
      "section": "the upshot",
      "display_name": "The Upshot"
    },
    {
      "section": "video",
      "display_name": "Video"
    },
    {
      "section": "the weekly",
      "display_name": "The Weekly"
    },
    {
      "section": "well",
      "display_name": "Well"
    },
    {
      "section": "world",
      "display_name": "World"
    },
    {
      "section": "your money",
      "display_name": "Your Money"
    }
  ]

list(map(lambda s: s["section"], sections))


['admin',
 'arts',
 'automobiles',
 'books',
 'briefing',
 'business',
 'climate',
 'corrections',
 'education',
 'en español',
 'fashion',
 'food',
 'gameplay',
 'guide',
 'health',
 'home & garden',
 'home page',
 'job market',
 'the learning network',
 'lens',
 'magazine',
 'movies',
 'multimedia/photos',
 'new york',
 'obituaries',
 'opinion',
 'parenting',
 'podcasts',
 'reader center',
 'real estate',
 'smarter living',
 'science',
 'sports',
 'style',
 'sunday review',
 't brand',
 't magazine',
 'technology',
 'theater',
 'times insider',
 'today’s paper',
 'travel',
 'u.s.',
 'universal',
 'the upshot',
 'video',
 'the weekly',
 'well',
 'world',
 'your money']